In [1]:
import json
import pandas as pd

In [6]:
def extract_face_data(predictions_json_path, model_type):
    # Load the JSON data from the file
    with open(predictions_json_path, 'r') as f:
        data = json.load(f)
    
    # Initialize a dictionary to hold all file data
    all_files_data = {}

    # Iterate through each file's data in the JSON
    for file_data in data:
        file_name = file_data['source']['filename']
        predictions = file_data['results']['predictions'][0]['models'][model_type]['grouped_predictions'][0]['predictions']

        # Create a DataFrame for the current file's predictions
        face_data = pd.DataFrame(predictions)
        
        emotions = [d['name'] for d in face_data['emotions'][0]]
        emotions_data = pd.DataFrame(face_data['emotions'].to_list(), columns=emotions)

        for emotion in emotions:
            emotions_data[emotion] = pd.json_normalize(emotions_data[emotion])['score']
        
        face_data = face_data.join(emotions_data)
        
        # Calculate the average score for each emotion
        average_emotions = emotions_data.mean().to_dict()
        
        # Store the average emotions data in the dictionary
        all_files_data[file_name] = average_emotions

    return all_files_data


def save_results_to_json(data, output_json_path):
    with open(output_json_path, 'w') as f:
        json.dump(data, f, indent=4)

In [10]:
models = ["face", "burst", "prosody"]

for model in models:
    predictions_json_path = f"raw_predictions/{model}_predictions.json"
    output_json_path = f"final_predictions/final_{model}_predictions.json"
    average_predictions = extract_face_data(predictions_json_path, model)
    save_results_to_json(average_predictions, output_json_path)
    print(average_predictions)

{'Sustainability Spotlight Darren Woods Chairman & CEO of ExxonMobil.mp4': {'Admiration': 0.06243617792168389, 'Adoration': 0.041046057098909565, 'Aesthetic Appreciation': 0.04559940921709589, 'Amusement': 0.09939111680116343, 'Anger': 0.08056642155608405, 'Anxiety': 0.23258778663433116, 'Awe': 0.1390761563392437, 'Awkwardness': 0.2322215138246184, 'Boredom': 0.3181382223315861, 'Calmness': 0.26469077744885633, 'Concentration': 0.3671891193674958, 'Confusion': 0.6090717843693235, 'Contemplation': 0.1839900964304157, 'Contempt': 0.14965129619383294, 'Contentment': 0.067796993676735, 'Craving': 0.04899386551393115, 'Desire': 0.045897356821629015, 'Determination': 0.09099825560722662, 'Disappointment': 0.3129669102313726, 'Disgust': 0.1407632413117782, 'Distress': 0.3058245209572108, 'Doubt': 0.4063053017725115, 'Ecstasy': 0.027936400420477858, 'Embarrassment': 0.12279789400813372, 'Empathic Pain': 0.13693238146927045, 'Entrancement': 0.10384926276848368, 'Envy': 0.041024534517656204, 'Ex

In [9]:
def extract_face_data(predictions_json_path, model_type='face'):

    with open(predictions_json_path, 'r') as f:
        data = json.load(f)
    
    all_files_data = {}

    for file_data in data:
        file_name = file_data['source']['filename']
        try:
            predictions = file_data['results']['predictions'][0]['models'][model_type]['grouped_predictions'][0]['predictions']
        except (IndexError, KeyError):
            # Handle the case where predictions are empty or the structure is not as expected
            all_files_data[file_name] = {}
            continue
        
        face_data = pd.DataFrame(predictions)
        
        if not face_data.empty:
            emotions = [d['name'] for d in face_data['emotions'][0]]
            emotions_data = pd.DataFrame(face_data['emotions'].to_list(), columns=emotions)

            for emotion in emotions:
                emotions_data[emotion] = pd.json_normalize(emotions_data[emotion])['score']
            
            face_data = face_data.join(emotions_data)
            
            # Calculate the average score for each emotion
            average_emotions = emotions_data.mean().to_dict()

            # Determine the top emotion
            top_emotion = max(average_emotions, key=average_emotions.get)
            average_emotions['top_emotion'] = top_emotion
            
            # Store the average emotions data in the dictionary
            all_files_data[file_name] = average_emotions
        else:
            all_files_data[file_name] = {}

    return all_files_data

def save_results_to_json(data, output_json_path):
    with open(output_json_path, 'w') as f:
        json.dump(data, f, indent=4)
